### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
import IPython.display
from IPython.display import Audio
import torch.optim as optim
from types import SimpleNamespace
import scipy.signal as sc
import time
from sklearn.metrics import f1_score
import pickle

from trainDataset import TrainDataset
from testDataset import TestDataset
#from trainDatasetNew import TrainDatasetNew
#from testDatasetNew import TestDatasetNew
from validation_split import get_dataloaders
from math_utils import logMagStft, ffts
from SpectrogramCNN import SpectrogramCNN
from train_utils import train, test, save_output, save_geometric_mean_predictions
from evaluation_utils import get_mean_F1
from MulitScale1DResNet import MSResNet
from LSTM import LSTM
from FFTMulitScale1DResNet import FFTMSResNet
from SpectralResNet import SpectralResNet34
from scipy.signal import hilbert
from FeatureFNN import FeatureFNN
from MelRawCombined import CNN1D, CNN2D

### Parameters

In [2]:
validation_split = .2
do_plots = False
load_model = True
args = SimpleNamespace(batch_size=64, test_batch_size=64, epochs=7,
                       lr=0.001, momentum=0.5, seed=1, log_interval=200, 
                      net = CNN2D) #SpectrogramCNN, MSResNet, SpectralResNet34, FFTMSResNet, FeatureFNN, CNN1D, CNN2D 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not torch.cuda.is_available(): # adapt those paths on other machine
    print('no cuda')
    path_train = './../data/train-small/'
    path_test =  './../data/test/kaggle-test/'
else:
    print('with cuda')
    path_train = './../data/kaggle-train/'
    path_test =  './../data/kaggle-test/'
    
path_model = 'models/model.pt'
path_submission = 'submissions/'
    
sample_rate = 16000
nmbr_classes = 10

with cuda


### Original Dataset

In [3]:
# todo add in the classes the features and the fft data

toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

trainDataset = TrainDataset(path_train, transform=toFloat)
print(len(trainDataset))

testDataset = TestDataset(path_test, transform=toFloat)
print(len(testDataset))

269776
4096


In [4]:
input_size = len(trainDataset[0][0])
print('input size: ',input_size)

input size:  64000


### Look at Original Data

In [5]:
if do_plots:
    # how many instruments are there?
    dummy_count = np.zeros(20)

    for sample in trainDataset:
        dummy_count[sample[1]] += 1

    labels_count = []
    for elem in dummy_count:
        if elem != 0:
            labels_count.append(elem)

    print(labels_count)

In [6]:
if do_plots:
    nmbr_classes = len(labels_count)
    print('nmbr_classes: ', nmbr_classes)

In [7]:
if do_plots:
    plt.plot(labels_count, '*')

In [8]:
if do_plots:
    # plot one of each

    done = np.zeros(nmbr_classes)
    examples = []

    for sample in trainDataset:
        if done[sample[1]] == 0:
            examples.append(sample)
            done[sample[1]] = 1

In [9]:
plt.rcParams["figure.figsize"] = (20,10)

In [10]:
#time series

if do_plots:

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        plt.plot(examples[i][0])

In [11]:
#envelope

if do_plots:
    
    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        env = torch.from_numpy(np.abs(hilbert(examples[i][0]))).view(1,1,-1)
        env = m(env)
        plt.plot(env[0,0].numpy())
        
    plt.show()

In [12]:
# fft

if do_plots:

    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        fft = torch.from_numpy(ffts(examples[i][0])).view(1,1,-1)
        fft = m(fft)
        plt.plot(fft[0,0].numpy())
        
    plt.show()

In [13]:
# melspec

if do_plots:
    n_fft = 510
    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        spectrogram = logMagStft(examples[i][0], 16000, n_fft)
        plt.imshow(spectrogram)
        
    plt.show()

In [14]:
if do_plots:

    for sample in examples:
        display(Audio(sample[0], rate=sample_rate))

### Dataloaders

In [15]:
# validation split is done here

train_loader, validation_loader, indices = get_dataloaders(trainDataset, 
                                                  batch_size = args.batch_size, 
                                                  validation_split = validation_split, 
                                                  shuffle_dataset = True, 
                                                  random_seed = None)
                                                    #indices = indices)

for samples, instrument_family_target in train_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break
        
for samples, instrument_family_target in validation_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break

torch.Size([64, 64000]) torch.Size([64]) tensor([0, 3, 4, 3, 6, 6, 9, 5, 3, 3, 5, 6, 8, 7, 2, 4, 3, 1, 4, 8, 4, 5, 1, 3,
        7, 6, 5, 4, 8, 5, 6, 4, 0, 0, 7, 7, 8, 8, 4, 7, 3, 0, 0, 7, 0, 6, 6, 4,
        4, 0, 0, 0, 5, 6, 6, 9, 5, 0, 8, 0, 8, 4, 3, 0])
tensor(-0.9551, dtype=torch.float64) tensor(0.9466, dtype=torch.float64)
['bass' 'guitar' 'keyboard' 'guitar' 'organ' 'organ' 'vocal' 'mallet'
 'guitar' 'guitar' 'mallet' 'organ' 'string' 'reed' 'flute' 'keyboard'
 'guitar' 'brass' 'keyboard' 'string' 'keyboard' 'mallet' 'brass' 'guitar'
 'reed' 'organ' 'mallet' 'keyboard' 'string' 'mallet' 'organ' 'keyboard'
 'bass' 'bass' 'reed' 'reed' 'string' 'string' 'keyboard' 'reed' 'guitar'
 'bass' 'bass' 'reed' 'bass' 'organ' 'organ' 'keyboard' 'keyboard' 'bass'
 'bass' 'bass' 'mallet' 'organ' 'organ' 'vocal' 'mallet' 'bass' 'string'
 'bass' 'string' 'keyboard' 'guitar' 'bass']
torch.Size([64, 64000]) torch.Size([64]) tensor([9, 3, 5, 0, 8, 6, 3, 0, 4, 0, 8, 8, 0, 4, 3, 0, 3, 3, 0, 4, 0, 5,

In [16]:
# save indices
with open(path_submission + "indices.txt", "wb") as fp:
    pickle.dump([indices], fp)

In [17]:
test_loader = data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False) #!!! shuffle should be false
for samples in test_loader:
        print(samples[0].shape)
        print(torch.min(samples[0]), torch.max(samples[0]))
        break

torch.Size([64, 64000])
tensor(-0.9723, dtype=torch.float64) tensor(0.9834, dtype=torch.float64)


### Main

In [18]:
if (args.net == SpectrogramCNN) or (args.net == SpectralResNet34) \
    or (args.net == FeatureFNN) or (args.net == CNN1D) \
    or (args.net == CNN2D):
    model = args.net(device).to(device)

In [19]:
if (args.net == MSResNet) or (args.net == FFTMSResNet):
    model = args.net(1, device).to(device)

In [20]:
if args.net == LSTM:
    model = args.net(device, input_size = 252, hidden_size = 512, num_layers = 1, num_classes = 10).to(device)

In [21]:
print(model)

CNN2D(
  (conv_1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 10), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(4, 10), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(4, 10), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_4): Sequential(
    (0): Conv2d(32, 32, kernel_size=(4, 10), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1,

In [22]:
# Main
optimizer = optim.Adam(model.parameters(), lr=args.lr)

info = {'highest F1' : 0,
        'saved epoch' : None}

In [23]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch, start_time = time.time())
    f1 = get_mean_F1(model, validation_loader)
    print('after epoch {} got f1 score of {}'.format(epoch , f1))
    if f1 > info['highest F1']:
        info['highest F1'] = np.copy(f1)
        info['saved epoch'] = epoch 
        test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)
        torch.save(model, path_model)
        print('currently best model --> saved')
        
print(info)

Train Epoch: 1 [0/269776 (0%)]	Loss: 2.303088	F1: 0.0625	Runtime: 0.7
Train Epoch: 1 [12800/269776 (6%)]	Loss: 1.333126	F1: 0.5000	Runtime: 56.6
Train Epoch: 1 [25600/269776 (12%)]	Loss: 1.117404	F1: 0.6406	Runtime: 112.5
Train Epoch: 1 [38400/269776 (18%)]	Loss: 1.005882	F1: 0.6719	Runtime: 168.2
Train Epoch: 1 [51200/269776 (24%)]	Loss: 0.947920	F1: 0.6250	Runtime: 224.0
Train Epoch: 1 [64000/269776 (30%)]	Loss: 0.718413	F1: 0.7812	Runtime: 279.7
Train Epoch: 1 [76800/269776 (36%)]	Loss: 0.641225	F1: 0.7500	Runtime: 335.5
Train Epoch: 1 [89600/269776 (42%)]	Loss: 0.520325	F1: 0.8281	Runtime: 391.2
Train Epoch: 1 [102400/269776 (47%)]	Loss: 0.676766	F1: 0.7344	Runtime: 447.1
Train Epoch: 1 [115200/269776 (53%)]	Loss: 0.665857	F1: 0.7500	Runtime: 502.8
Train Epoch: 1 [128000/269776 (59%)]	Loss: 0.421030	F1: 0.8594	Runtime: 558.5
Train Epoch: 1 [140800/269776 (65%)]	Loss: 0.653633	F1: 0.7344	Runtime: 614.3
Train Epoch: 1 [153600/269776 (71%)]	Loss: 0.803640	F1: 0.7188	Runtime: 669.9
Tra

Train Epoch: 6 [204800/269776 (95%)]	Loss: 0.201684	F1: 0.9219	Runtime: 901.1
after epoch 6 got f1 score of 0.33443794431279666
Train Epoch: 7 [0/269776 (0%)]	Loss: 0.024902	F1: 1.0000	Runtime: 0.4
Train Epoch: 7 [12800/269776 (6%)]	Loss: 0.027976	F1: 1.0000	Runtime: 56.5
Train Epoch: 7 [25600/269776 (12%)]	Loss: 0.098622	F1: 0.9531	Runtime: 112.6
Train Epoch: 7 [38400/269776 (18%)]	Loss: 0.094430	F1: 0.9844	Runtime: 168.6
Train Epoch: 7 [51200/269776 (24%)]	Loss: 0.089394	F1: 0.9688	Runtime: 224.6
Train Epoch: 7 [64000/269776 (30%)]	Loss: 0.015583	F1: 1.0000	Runtime: 280.6
Train Epoch: 7 [76800/269776 (36%)]	Loss: 0.043169	F1: 0.9844	Runtime: 336.5
Train Epoch: 7 [89600/269776 (42%)]	Loss: 0.058544	F1: 0.9844	Runtime: 392.5
Train Epoch: 7 [102400/269776 (47%)]	Loss: 0.022424	F1: 1.0000	Runtime: 448.4
Train Epoch: 7 [115200/269776 (53%)]	Loss: 0.027738	F1: 0.9844	Runtime: 505.3
Train Epoch: 7 [128000/269776 (59%)]	Loss: 0.011709	F1: 1.0000	Runtime: 561.2
Train Epoch: 7 [140800/269776 (

### save or load outputs

In [19]:
save_output(args, model, device, test_loader, 'CNN2D', 
            trainDataset, testDataset, path_save = path_submission)

NameError: name 'fieldnames' is not defined

### Load Model

In [18]:
if load_model:
    #model = torch.load(path_model)
    model = torch.load('models/model_7epoch_CNN2D.pt')

In [ ]:
# load indices
with open(path_submission + "indices.txt", "rb") as fp:
    indices = pickle.load(fp)

In [ ]:
get_mean_F1(model, validation_loader)

In [ ]:
epoch=10
test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)